In [2]:
import numpy as np
import pandas as pd
import hvplot.pandas

In [3]:
# Set the random seed for reproducibility
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [4]:
# Load the fear and greed sentiment data for Bitcoin
df = pd.read_csv('btc_sentiment.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
df = df.drop(columns="fng_classification")
df.head()

C:\Users\Dung\AppData\Local\Temp\ipykernel_9908\967410264.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv('btc_sentiment.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
C:\Users\Dung\AppData\Local\Temp\ipykernel_9908\967410264.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv('btc_sentiment.csv', index_col="date", infer_datetime_format=True, parse_dates=True)


,fng_value
date,
2019-07-29,19
2019-07-28,16
2019-07-27,47
2019-07-26,24
2019-07-25,42


In [5]:
# Load the historical closing prices for Bitcoin
df2 = pd.read_csv('btc_historic.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)['Close']
df2 = df2.sort_index()
df2.tail()

C:\Users\Dung\AppData\Local\Temp\ipykernel_9908\3705894443.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df2 = pd.read_csv('btc_historic.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)['Close']


Date
2019-07-25    9882.429688
2019-07-26    9847.450195
2019-07-27    9478.320313
2019-07-28    9531.769531
2019-07-29    9529.889648
Name: Close, dtype: float64

In [6]:
# Join the data into a single DataFrame
df = df.join(df2, how="inner")
df.tail()

,fng_value,Close
date,,
2018-02-05,11,6937.080078
2018-02-04,24,8218.049805
2018-02-03,40,9251.269531
2018-02-02,15,8870.820313
2018-02-01,30,9114.719727


In [7]:
df.head()

,fng_value,Close
date,,
2019-07-29,19,9529.889648
2019-07-28,16,9531.769531
2019-07-27,47,9478.320313
2019-07-26,24,9847.450195
2019-07-25,42,9882.429688


In [8]:
# This function accepts the column number for the features (X) and the target (y)
# It chunks the data up with a rolling window of Xt-n to predict Xt
# It returns a numpy array of X any y
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [9]:
# Predict Closing Prices using a 3 day window of previous fng values
window_size = 3

# Column index 0 is the 'fng_value' column
# Column index 1 is the `Close` column
feature_column = 0
target_column = 1
X, y = window_data(df, window_size, feature_column, target_column)

In [10]:
# Use 70% of the data for training and the remaineder for testing
split = int(0.7 * len(X))
X_train = X[: split]
X_test = X[split:]
y_train = y[: split]
y_test = y[split:]


In [11]:
# Use the MinMaxScaler to scale data between 0 and 1.
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X)

MinMaxScaler()

In [12]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

scaler.fit(y)

y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)

In [13]:
# Reshape the features for the model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [15]:
# Define the LSTM RNN model.
model = Sequential()

number_units = 50
dropout_fraction = 0.2

# Layer 1
model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train.shape[1], 1))
    )
model.add(Dropout(dropout_fraction))
# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))
# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))
# Output layer
model.add(Dense(1))

In [16]:
model.compile(optimizer="adam", loss="mean_squared_error")

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 3, 50)             10400     
                                                                 
 dropout (Dropout)           (None, 3, 50)             0         
                                                                 
 lstm_1 (LSTM)               (None, 3, 50)             20200     
                                                                 
 dropout_1 (Dropout)         (None, 3, 50)             0         
                                                                 
 lstm_2 (LSTM)               (None, 50)                20200     
                                                                 
 dropout_2 (Dropout)         (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 5

In [18]:
# Train the model
model.fit(X_train, y_train, epochs=15, shuffle=False, batch_size=1, verbose=1)

Epoch 1/15
377/377 [==============================] - 16s 22ms/step - loss: 0.0401
Epoch 2/15
377/377 [==============================] - 8s 22ms/step - loss: 0.0108
Epoch 3/15
377/377 [==============================] - 8s 22ms/step - loss: 0.0087
Epoch 4/15
377/377 [==============================] - 8s 22ms/step - loss: 0.0108
Epoch 5/15
377/377 [==============================] - 8s 22ms/step - loss: 0.0111
Epoch 6/15
377/377 [==============================] - 8s 22ms/step - loss: 0.0135
Epoch 7/15
377/377 [==============================] - 8s 22ms/step - loss: 0.0160
Epoch 8/15
377/377 [==============================] - 8s 22ms/step - loss: 0.0201
Epoch 9/15
377/377 [==============================] - 8s 22ms/step - loss: 0.0165
Epoch 10/15
377/377 [==============================] - 8s 22ms/step - loss: 0.0238
Epoch 11/15
377/377 [==============================] - 8s 22ms/step - loss: 0.0215
Epoch 12/15
377/377 [==============================] - 8s 22ms/step - loss: 0.0261
Epoch 13/15


## Model Performance

In [19]:
# Evaluate the model
model.evaluate(X_test, y_test)

6/6 [==============================] - 2s 13ms/step - loss: 0.0550


0.05495327338576317

In [20]:
# Make some predictions
predicted = model.predict(X_test)

6/6 [==============================] - 2s 10ms/step


In [21]:
# Recover the original prices instead of the scaled version
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

In [22]:
# Create a DataFrame of Real and Predicted values
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
}, index = df.index[-len(real_prices): ]) 
stocks.head()

,Real,Predicted
date,,
2018-07-15,6740.549805,6352.361328
2018-07-14,6364.259766,6354.243652
2018-07-13,6268.750000,6356.622070
2018-07-12,6229.830078,6359.106934
2018-07-11,6253.600098,6361.111328


In [23]:
stocks.hvplot()

:NdOverlay   [Variable]
   :Curve   [date]   (value)